DATA SCIENCE I ASSIGNMENT 2

In [9]:
# Import libraries 
import pandas as pd 

In [4]:
# Import dataset
df = pd.read_csv("experiment_dataset.csv")
df

,Unnamed: 0,Age,Location,Device,Variant,Time Spent,CTR
0,0,62,Location2,Device2,Control,13.928669,0.084776
1,1,18,Location1,Device1,Variant B,11.310518,0.096859
2,2,21,Location2,Device1,Variant B,24.842100,0.097630
3,3,21,Location1,Device3,Variant B,20.061300,0.109783
4,4,57,Location1,Device2,Variant B,34.495503,0.068579
...,...,...,...,...,...,...,...
995,995,39,Location2,Device2,Variant B,17.252030,0.092211
996,996,38,Location3,Device2,Control,30.075898,0.078151
997,997,60,Location2,Device3,Control,31.929223,0.125213
998,998,35,Location2,Device2,Variant B,14.680299,0.095423


1. Analyze the results to determine which feature (if any) results in CTR or Time Spent lift.

We can determine this through first isolating the data by feature variants (Control, Variant A, Variant B) and then comparing them to the mean CTR and Time Spent for each Variant. 

In [5]:
# Group the data by variant and calculate mean of CTR and Time Spent for each variant
grouped_data = df.groupby('Variant').agg({'CTR': 'mean', 'Time Spent': 'mean'}).reset_index()

# Display the grouped data
print(grouped_data)

     Variant       CTR  Time Spent
0    Control  0.098554   20.070781
1  Variant A  0.120269   24.805547
2  Variant B  0.108933   23.343783


Using the groupby function, we can see the mean CTR and Time Spent for each variant. However, in order to determine if there is a significant lift in CTR or Time Spent with either of the feature variants compared to the control, we need to look for a higher mean value in these matrics. 

*Note: Lift is calculated as the percentage incrase in the metric, compared to the control group.

In [7]:
# Calculate lift
grouped_data['CTR Lift'] = (grouped_data['CTR'] - grouped_data.loc[grouped_data['Variant'] == 'Control', 'CTR'].values[0]) / grouped_data.loc[grouped_data['Variant'] == 'Control', 'CTR'].values[0]
grouped_data['Time Spent Lift'] = (grouped_data['Time Spent'] - grouped_data.loc[grouped_data['Variant'] == 'Control', 'Time Spent'].values[0]) / grouped_data.loc[grouped_data['Variant'] == 'Control', 'Time Spent'].values[0]

# Display the grouped data with lift
print(grouped_data)

     Variant       CTR  Time Spent  CTR Lift  Time Spent Lift
0    Control  0.098554   20.070781  0.000000         0.000000
1  Variant A  0.120269   24.805547  0.220339         0.235903
2  Variant B  0.108933   23.343783  0.105312         0.163073


Based on the results, the following interpretations can be made: 

Variant A: 
- The CTR improved by 22.03% compared to the control group 
- Time Spent improved by 23.59% compared to the control group

Variant B: 
- The CTR improved by 10.53% compared to the control group
- Time Spent improved by 16.31% compared to the control group

Therefore, the analysis shows that both Variant A and B resulted in a CTR and Time Spent lift compared to the control group. Given the data, variant A seems to be the most promising feature variant, providing a higher lift in both CTR and Time Spent compared to Variant B.

2. Conduct statistical testing to determine if there is a statistically significant difference between the features and the control group.

A/B testing scenario compares the means of two groups to determine if they're significantly different. We can perform this using the stats package from scipy library. 

Test Set-up: 
- Null Hypothesis (H0): There is no difference in CTR or Time Spent between the control group and the variant groups (A and B).
- Alternative Hypothesis (H1): There is a difference in CTR or Time Spent between the control group and the variant groups (A and B).

In [12]:
from scipy.stats import ttest_ind

# Isolate data for each variant
control = df[df['Variant'] == 'Control']
variant_A = df[df['Variant'] == 'Variant A']
variant_B = df[df['Variant'] == 'Variant B']

# Perform t-test for CTR between Control and Variant A
t_stat, p_value = ttest_ind(control['CTR'], variant_A['CTR'])
print("CTR t-test: Control vs Variant A: t_stat = ", t_stat, ", p-value = ", p_value)

# Perform t-test for Time Spent between Control and Variant A
t_stat, p_value = ttest_ind(control['Time Spent'], variant_A['Time Spent'])
print("Time Spent t-test: Control vs Variant A: t_stat = ", t_stat, ", p-value = ", p_value)

# Perform t-test for CTR between Control and Variant B
t_stat, p_value = ttest_ind(control['CTR'], variant_B['CTR'])
print("CTR t-test: Control vs Variant B: t_stat = ", t_stat, ", p-value = ", p_value)

# Perform t-test for Time Spent between Control and Variant B
t_stat, p_value = ttest_ind(control['Time Spent'], variant_B['Time Spent'])
print("Time Spent: for Control vs Variant B: t_stat = ", t_stat, ", p-value = ", p_value)


CTR t-test: Control vs Variant A: t_stat =  -13.829424737499187 , p-value =  1.9602781373243157e-38
Time Spent t-test: Control vs Variant A: t_stat =  -12.142363487472364 , p-value =  8.488565644996449e-31
CTR t-test: Control vs Variant B: t_stat =  -6.4718143491783255 , p-value =  1.8743198199982106e-10
Time Spent: for Control vs Variant B: t_stat =  -8.174237395991806 , p-value =  1.496358076285182e-15


Based on the results we can interpret the following: 
Variant A: 
- For CTR, the null hypothesis can be rejected as the p-value is far less than the 0.05 significance level therefore suggesting that we have statistically significant evidence to suggest that Variant A results in a higher CTR compared to the control 
- For Time Spent, the null hypothesis can also be rejected for a reason similar to above ^

Variant B: 
- For CTR, we can reject the null hypothesis as the p-value is far less than the 0.05 significance level therefore suggesting that we have statistically significant evidence to suggest that Variant B results in a higher CTR compared to the control 
- For Time Spent, the null hypothesis can also be rejected for a reason similar to above ^

For all tests, the p-value is less than 0.05 (in fact, much smaller), which means that we can reject the null hypothesis (the assumption that there is no difference between the groups). The negative t-statistic score indicates that the mean for the variant groups is higher than the mean for the control group, aligning with the lift observed earlier.

3. Summarize your results. Make a recommendation to the engineering team about which feature to deploy. 

A data analysis was performed on the experiment results in two phases. The first phase was a descriptive analysis where both Variant A and B showed an increase in Click-Through-Rate (CTR) and Time Spent in comparison to the control group. Variant A showed a 22.03% increase in CTR and a 23.59% increase in Time Spent while variant B resulted in a 10.53% increase in CTR and a 16.31% increase in Time Spent. The second phase was statistical testing, during which the t-test scores and p-values were calculated in order to determine whether the null hypothesis that there is no difference in CTR or Time Spent between the control group and the variant groups should be rejected or not. Based on both variants t-test results, there was strong evidence to reject the null hypothesis. For both Variant A and B, there were statistically significant increases in CTR and Time Spent as compared to the Control Group. However, Variant A did show a larger effect size than Variant B, indicating a higher lift in both CTR and Time Spent. 

Therefore, based on the results I would recommend the engineering team to consider deploying Variant A. This variant not only led to a statistically significant increase in both key metrics of CTR and Time Spent, but also demonstrated a larger effect size compared to Variant B, meaning the improvements were more substatial perhaps. However, it is also important to consider potential impact on other parts of the product and user experience. The engineering team should ensure that this change aligns well with the overall user experience and does not negatively impact other features or functionalities.

4. Create a roll-out plan. How quickly will you introduce the feature to your audience?

1) Polit Rollout
We should first consider deploying the feature to a small percentage of the user base. This could be about 1 - 5 percent of our users. What this phase will allow us to do is monitor the new feature in the active/live environment with a smaller set of users and therefore easily catch any unexpected issues that were not identified during testing. 

2) Phased Rollout 
Once the pilot rollout is a success and does not show any major issues, we can then begin to gradually increase the percentage of new users who receive this new feature. The user increase plan can look something like this: 
- Week 1: Extend to 10% users 
- Week 2: Extend to 25% users 
- Week 3: Extend to 50% users 
- Week 4: Extend to 74% users
During this phease, we can continue to monitor the performance of the new feature while collecting user feedback to ensure that the observed improvements from our analysis are still persisting. 

*Note: The specific percentages and timeline for the phased rollout can be adjusted based on the team's resources, the complexity of the feature, the size and diversity of your user base, and other factors. This plan is a general guideline and may need to be further customized based on needs, preferences, and resources.

3) Complete rollout 
Finally, if the phased rollout goes well we can deploy the feature to 100% of our userbase. 

Throughout this plan, we need to ensure that there are strong mechanisms set in place for collecting user feedback and closely monitoring key metrics to ensure there is no negative impact. If any issues are detected during the pilot or rollout phase, we should be prepared to roll back the feature, fix the issues, and then resume the rollout plan.